In [1]:
# All shortest paths calculated by Algorithm 8 and 9 may be incomplete.
# Here is an example of four nodes.


In [2]:
exec(open("./funcs/tool_funcs.py").read())



In [3]:
def dijkstra_show_path(adj_matrix, start, end):
 
    n = len(adj_matrix)
    distances = [float('inf')] * n
    previous_nodes = [-1] * n
    distances[start] = 0
    priority_queue = [(0, start)]  # (distance, node)

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # Stop if we reached the destination node
        if current_node == end:
            break

        # If the distance is no longer optimal, skip
        if current_distance > distances[current_node]:
            continue

        # Explore neighbors
        for neighbor, weight in enumerate(adj_matrix[current_node]):
            if weight > 0:  # Only consider edges with weight > 0
                distance = current_distance + weight

                # Update distance if it's better
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node
                    heapq.heappush(priority_queue, (distance, neighbor))

    # Reconstruct the path from end to start
    path = []
    current = end
    while current != -1:
        path.append(current)
        current = previous_nodes[current]
    path.reverse()

    # If the start node isn't reachable, return an empty path
    return path if path[0] == start else []

In [4]:
def translate_path(path, candidate_node_list):
    new_path = [candidate_node_list[i] for i in path]
    return new_path




In [5]:
def output_small_matrix(X_distance_matrix, candidate_node_list):
    small_matrix = np.zeros((len(candidate_node_list), len(candidate_node_list)))
    for i, ii in enumerate(candidate_node_list):
        for j, jj in enumerate(candidate_node_list):
            small_matrix[i,j] = X_distance_matrix[ii, jj]
    return small_matrix




In [6]:
def cal_candidate_node_list(i, j, shortest_path_matrix, X_distance_matrix):
    
    N = len(X_distance_matrix)
    
    remaining_list = [k for k in np.arange(N) if k != i and k != j]
    candidate_node_list = []
    candidate_node_list.append(i)
    for t in remaining_list:
        if shortest_path_matrix[i,j] < shortest_path_matrix[i,t] + shortest_path_matrix[t,j]:
            pass
        else:
            candidate_node_list.append(t)
    candidate_node_list.append(j)
 
    return candidate_node_list
        



In [7]:

def cal_path_warm_start(i, j, shortest_path_matrix, X_distance_matrix):
    candidate_node_list = cal_candidate_node_list(i, j, shortest_path_matrix, X_distance_matrix)
    print("candidate_node_list:")
    print(candidate_node_list)
   
    small_matrix = output_small_matrix(X_distance_matrix, candidate_node_list)
    path = dijkstra_show_path(small_matrix, 0, len(candidate_node_list) - 1)
    path = translate_path(path, candidate_node_list)
    
    return path


In [8]:
# APPD stands for all points path distance (APPD).

def check_shortest_path(i,j, X_distance_matrix, APPD_matrix, path):
    temp1= path[:-1]
    temp2= path[1:]
    llll = [X_distance_matrix[temp1[i],temp2[i]] for i in range(len(temp1))]
    ooo = np.sum(llll)
    return ooo == APPD_matrix[i,j]

In [9]:
def check_if_temp_path_already_in(all_paths_list, temp_path):
    aa = set(temp_path)
    for bb in all_paths_list:
        bb = set(bb)
        if aa == bb:
            return True
    return False

def cal_all_paths_warm_start(i, j, shortest_path_matrix, X_distance_matrix):
    candidate_node_list = cal_candidate_node_list(i, j, shortest_path_matrix, X_distance_matrix)
 
    small_matrix = output_small_matrix(X_distance_matrix, candidate_node_list)
    path = dijkstra_show_path(small_matrix, 0, len(candidate_node_list) - 1)

    all_paths_list = []
    all_paths_list.append(path)

    K = len(candidate_node_list)
    # temp_list = [q for q in range(K) if q not in path]
    
    temp_list = [q for q in range(1, K - 1)]

    for m in temp_list:
        temp_path1 = dijkstra_show_path(small_matrix, 0, m)
        temp_path2 = dijkstra_show_path(small_matrix, m, K - 1)
        temp_path =  temp_path1[:-1] + temp_path2
        if not check_if_temp_path_already_in(all_paths_list, temp_path):
            all_paths_list.append(temp_path)

    all_paths_list = [translate_path(ppp, candidate_node_list) for ppp in all_paths_list]    

    return all_paths_list

In [10]:
# N = 1000

# X_distance_matrix = create_distance_matrix(N)

# X_distance_matrix = np.loadtxt("./data/X_100_distance_matrix.csv", delimiter=",")


In [11]:
X_distance_matrix = np.array([[  0., 5., 12., np.inf],
       [np.inf,   0.,  7., 10.],
       [np.inf,   np.inf,  0., 3.],
       [np.inf,  np.inf,   np.inf, 0.]])

In [12]:



shortest_path_matrix = floyd_warshall(X_distance_matrix)
 

In [13]:

N = len(X_distance_matrix)


In [14]:
i = 0
j = 3
 
path1 = dijkstra_show_path(X_distance_matrix, i, j)
    
path2 = cal_path_warm_start(i, j, shortest_path_matrix, X_distance_matrix)   
print(path1)
print(path2)


    

candidate_node_list:
[0, 1, 2, 3]
[0, 1, 3]
[0, 1, 3]


In [15]:
check_shortest_path(i,j, X_distance_matrix, shortest_path_matrix, path2)



True

In [16]:
all_paths_list = cal_all_paths_warm_start(i, j, shortest_path_matrix, X_distance_matrix)


In [17]:
for path_i in all_paths_list:
    aaa = check_shortest_path(i,j, X_distance_matrix, shortest_path_matrix, path_i)
    print(path_i, aaa)



[0, 1, 3] True
[0, 2, 3] True


In [18]:
shortest_path_matrix[0, 3]

15.0

In [19]:
check_shortest_path(i,j, X_distance_matrix, shortest_path_matrix, [0, 1, 3])


True

In [20]:
check_shortest_path(i,j, X_distance_matrix, shortest_path_matrix, [0, 2, 3])

True

In [21]:
check_shortest_path(i,j, X_distance_matrix, shortest_path_matrix, [0, 1, 2, 3])


True

In [22]:
# Path [0, 1, 2, 3] is missed in all_paths_list.

# However, as indicated in Section 4.4 (Maintaining a key_node_list), 
# this does not affect the correctness of Algorithm 10 (warm-start calculation of APSP by key_node_list). 
# Because if all_paths_list is not complete, 
# the calculated key_node_list is a superset of the actual key_node_list, 
# this property makes it do not affect the correctness of Algorithm 10. 
 
#  A set A is a superset of another set B if all elements of the set B are elements of the set A.
